<a href="https://colab.research.google.com/github/mkbahk/AzureQuantum/blob/main/External_Access_to_Azure_Quantum_PASQAL_mkbahk_20250624.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#!pip install git+https://github.com/microsoft/azure-quantum-python.git

  Cloning https://github.com/microsoft/azure-quantum-python.git to /tmp/pip-req-build-th49q0v6
  Running command git clone --filter=blob:none --quiet https://github.com/microsoft/azure-quantum-python.git /tmp/pip-req-build-th49q0v6
  Resolved https://github.com/microsoft/azure-quantum-python.git to commit 57cf7aa8cdb8a1447b38cf9517975c95df9f92cc
ERROR: git+https://github.com/microsoft/azure-quantum-python.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [1]:
!pip install -U pip
!pip install qiskit[visualization]==1.4.3
!pip install azure-quantum[qiskit]==3.1.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.6 MB/s eta 0:00:00
  DEPRECATION: Building 'pylatexenc' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pylatexenc'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for pylatexenc: filename=p

In [1]:
!pip install pulser-simulation #Only for using the local Qutip emulator included in Pulser
!pip install pulser-core

In [2]:
import pulser
from pulser.devices import DigitalAnalogDevice
from pulser.waveforms import ConstantWaveform
from azure.quantum import Workspace
import json

# Azure Quantum 작업 공간 설정
connection_string = ""
workspace = Workspace.from_connection_string(connection_string)

# PASQAL 시뮬레이터 타겟 설정
target = workspace.get_targets(name="pasqal.sim.emu-tn")

In [3]:
# 2개의 큐bit를 위한 레지스터 정의 (2D 격자)
coords = [(0, 0), (10, 0)]  # 10μm 간격으로 두 원자 배치
register = pulser.Register.from_coordinates(coords, prefix="q")

# 펄스 시퀀스 생성
seq = pulser.Sequence(register, DigitalAnalogDevice)

# 간단한 펄스: 모든 큐bit에 Rydberg 상태로의 전이를 유도하는 Constant 펄스
seq.declare_channel("ch0", "rydberg_global")
# pulse = ConstantWaveform(duration=1000, value=2.0)  # 1μs 동안 일정한 진폭 2 rad/μs
from pulser.pulse import Pulse # Import Pulse from pulser.pulse
pulse = Pulse.ConstantPulse(duration=1000, amplitude=2.0, detuning=0.0, phase=0.0) # Create a Pulse object with detuning and phase
seq.add(pulse, "ch0")

In [4]:
# 펄스 시퀀스를 JSON 형식으로 변환
def prepare_input_data(seq):
    input_data = {}
    input_data["sequence_builder"] = json.loads(seq.to_abstract_repr())
    return json.dumps(input_data)
###def

In [5]:
# 작업 제출
job = target.submit(
    input_data=prepare_input_data(seq),
    input_data_format="pasqal.pulser.v1",
    output_data_format="pasqal.pulser-results.v1",
    name="KOLON-Simple PASQAL Simulation",
    shots=100
)

In [8]:
# 작업 완료 대기 및 결과 가져오기
job.wait_until_completed()
result = job.get_results()
print("\nSimulation Results:\n", result)


Simulation Results:
 {'10': 52, '01': 41, '11': 6, '00': 1}
